In [ ]:
import requests
import json
from datetime import datetime
from dateutil import parser
import pytz

utc=pytz.UTC
url = 'https://slab-77654.bubbleapps.io/version-test/api/1.1/obj/Alerts'
API_KEY = "768841b035ef72b86e7c3bbcd67ff245"
log_file_path = "/Users/sudipkar/Library/CloudStorage/GoogleDrive-sudip.kar@berkeley.edu/My Drive/295 Capstone/fast5.log"

def send_log_to_bubble(Date,AlertDesc,Priority,ip) :
    hed = {
    'Authorization': 'Bearer ' + API_KEY
    }
    myobj = {
        "AlertDesc": AlertDesc,
        "Date": Date,
        "Email": "sudip.kar@outlook.com",
        "IDS Name": "PiSense",
        "KitID": "Pi4 Model B",
        "DeviceId": "BCM2711",
        "Response": "no",
        "Priority":Priority,
        "Ip":ip
    }
    bubble_ips = get_ips_from_bubble()
    #print(ip)
    if(ip not in bubble_ips):
        req = requests.post(url, headers=hed, data=myobj)
        print("Log entry sent successfully.")
    else:
        print("Log already exists.")

def convert_for_bubble(log_entry,bubble_max_date):
    log_format = '%m/%d/%Y-%H:%M:%S'
    #print(log_entry)
    x = log_entry.split("[")
    y = log_entry.split(" ")
    Alert = x[4].strip("Classification: ")
    AlertDesc = Alert.rstrip(Alert.strip("Classification: ")[-1])
    Date = x[0]
    Priority = x[5].split("]",1)[0].strip("Priority: ")
    #print(bubble_ips)
    if (int(Priority)<4 and y[5]=='ATTACK_RESPONSE' and utc.localize(datetime.strptime(Date.split(".")[0],log_format))>parser.parse(bubble_max_date)):
        ip = x[5].split("{TCP} ",1)[1].split("->")
        send_log_to_bubble(Date, AlertDesc,int(Priority),ip[0].rsplit(":")[0].strip())

def get_ips_from_bubble():
    hed = {
    'Authorization': 'Bearer ' + API_KEY
    }
    req = requests.get(url, headers=hed)
    req_json = req.json()
    req_list = req_json.get('response', {}).get('results')
    bubble_ips = []
    for i in range(len(req_list)):
        bubble_ips.append(req_list[i].get('Ip'))
    #print (bubble_ips)
    return bubble_ips

def get_max_date_from_bubble():
    hed = {
    'Authorization': 'Bearer ' + API_KEY
    }
    req = requests.get(url, headers=hed)
    req_json = req.json()
    req_list = req_json.get('response', {}).get('results')
    bubble_dates = []
    for i in range(len(req_list)):
        bubble_dates.append(req_list[i].get('Date'))
    return max(bubble_dates)

with open(log_file_path, "r") as file:
        bubble_max_date = get_max_date_from_bubble()
        #print(bubble_max_date)
        for line in file:
            # Assuming each line is a log entry, format it as needed
            log_entry = line.strip()
            #Parse out suspicious activity and send only those to bubble after parsing needed data
            if "Classification: Not Suspicious Traffic" not in log_entry and "Classification: (null)" not in log_entry and "Classification: Misc activity" not in log_entry:
                convert_for_bubble(log_entry,bubble_max_date)